In [11]:
import pandas as pd
import requests
import json

In [61]:
df = pd.read_pickle('Data')

In [62]:
df.shape

(22112, 103)

In [63]:
df.head()

,Id,DataSourceId,DataId,MLSNumber,ModificationDateTime,StatusChangeDateTime,Status,StatusEnum,SaleType,SaleTypeEnum,...,ElementarySchoolDistrict,MiddleOrJuniorSchool,MiddleOrJuniorSchoolDistrict,HighSchool,HighSchoolDistrict,dataSourceName,dataSourceDescription,Location,lat,lng
0,10000,3,587798,S708420,2013-01-01 11:29:07,2013-01-01 11:29:07,S,0,NOD,3.0,...,None,None,None,None,None,None,None,None,33.747795,-117.863906
1,10001,3,615367,P835779,2013-01-01 10:00:56,2013-01-01 10:00:56,S,4,STD,1.0,...,None,None,None,None,None,None,None,POINT(34.0737809 -117.98583),34.073781,-117.985830
2,10002,3,616798,M12118580,2013-01-01 09:57:43,2013-01-01 09:57:43,S,4,"NOD,SPAY",3.0,...,None,None,None,None,None,None,None,POINT(34.576074 -118.00609),34.576074,-118.006090
3,10003,3,619145,H12113661,2013-01-01 07:45:48,2013-01-01 07:45:48,S,4,STD,1.0,...,None,None,None,None,None,None,None,POINT(34.033266 -117.817744),34.033266,-117.817744
4,10004,3,620751,BB12131404,2013-01-01 07:31:02,2013-01-01 07:31:02,S,4,STD,1.0,...,None,None,None,None,None,None,None,POINT(34.217548 -118.359678),34.217548,-118.359678


# Geocoding records without latitute and longtitute data

In [64]:
df_null = df[df.lat == 0]

In [65]:
df_null.shape

(33, 103)

### In data, we have 33 record missing latitute and longtitute data

In [66]:
df_null['FullAddress'] = df_null['Address'] + ',' + df_null['City'] + ',' + df_null['State']

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [67]:
df_null.FullAddress

1122                12435 Bay Hill Court,Garden Grove,CA
4626     8770 Half - Pinecrest Place,Rancho Cucamonga,CA
4776                     17605 Caprino Place,Van Nuys,CA
6850                   3114 GRANGEMOUNT Road,Glendale,CA
7609                   451 DELWOOD Court,Newbury Park,CA
8765             3848 SAN CLEMENTE Court,Newbury Park,CA
10119                 3888 HARTUNG Court,Newbury Park,CA
10159             4340 CAMINO DE LA ROSA,Newbury Park,CA
10241                    4989 VIA MESITA,Newbury Park,CA
10291                      4244 VIA AZUL,Newbury Park,CA
10396             3776 CALLE LINDA VISTA,Newbury Park,CA
10480               41 LOS VIENTOS Drive,Newbury Park,CA
10532              143 MONTENEGRO Circle,Newbury Park,CA
10779              3931 SAN MARCOS Court,Newbury Park,CA
10942           1188 GALESMORE Court,Westlake Village,CA
11005                824 ALDERDALE Court,Newbury Park,CA
11883               1611 Jennings (3rd) Street,Madera,CA
12092              21274 Ridged

## Find geocode by address

In [70]:
url = 'https://maps.googleapis.com/maps/api/geocode/json'
for index, row in df_null.iterrows():
    params = {'key': 'YanKrisWu', 'address': row['FullAddress']}
    try:
        r = requests.get(url, params=params)
        results = r.json()['results']
        location = results[0]['geometry']['location']
        # insert into original data
        df['lat'][index] = location['lat']
        df['lng'][index] = location['lng']
    except IndexError:
        print('error:', index)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


error: 4626
error: 6850
error: 7609
error: 8765
error: 10119
error: 10241
error: 10291
error: 10396
error: 10779
error: 10942
error: 11005
error: 11883
error: 12092
error: 12397
error: 12665
error: 14442
error: 15995
error: 17725
error: 19079
error: 21849


In [78]:
df[df.lat == 0].shape

(20, 103)

## We replace the lat and lng for 13 missing values. Still 20 records could not find geocode.

In [79]:
df.to_pickle('Data_2')